# Exploratory Data Analysis with SQL

## SpaceX Launch Data Analysis

This notebook performs exploratory data analysis on SpaceX launch data using SQL queries. We will analyze various aspects of the launches including launch sites, payload mass, landing outcomes, and success rates.

## 1. Setup and Data Loading

First, we'll load the necessary libraries and set up our SQL environment.

In [ ]:
# Load SQL extension for Jupyter
%load_ext sql

In [ ]:
# Import required libraries
import csv
import sqlite3
import prettytable
import pandas as pd

prettytable.DEFAULT = 'DEFAULT'

# Create SQLite database connection
con = sqlite3.connect("my_data1.db")
cur = con.cursor()

In [ ]:
# Connect SQL magic to the SQLite database
%sql sqlite:///my_data1.db

In [ ]:
# Load the SpaceX launch data from CSV file
df = pd.read_csv('spacex_launch_data.csv')
df.head()

In [ ]:
# Load the dataframe into the SQLite database table
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False)

90

## 2. Data Exploration

Let's start by exploring the structure and content of our data.

In [ ]:
# Display the column names and data types in the table
%sql PRAGMA table_info('SPACEXTBL')

 * sqlite:///my_data1.db
Done.


cid,name,type,notnull,dflt_value,pk
0,FlightNumber,INTEGER,0,None,0
1,Date,TEXT,0,None,0
2,BoosterVersion,TEXT,0,None,0
3,PayloadMass,REAL,0,None,0
4,Orbit,TEXT,0,None,0
5,LaunchSite,TEXT,0,None,0
6,Outcome,TEXT,0,None,0
7,Flights,INTEGER,0,None,0
8,GridFins,INTEGER,0,None,0
9,Reused,INTEGER,0,None,0


In [ ]:
# Display the names of the unique launch sites in the space mission
%sql SELECT DISTINCT LaunchSite FROM SPACEXTBL

 * sqlite:///my_data1.db
Done.


LaunchSite
CCSFS SLC 40
VAFB SLC 4E
KSC LC 39A


### Task 1: Display 5 records from a specific launch site

In [ ]:
# Display 5 records where launch sites begin with 'CCSFS SLC 40'
%sql SELECT * FROM SPACEXTBL WHERE LaunchSite LIKE 'CCSFS SLC 40%' LIMIT 5

 * sqlite:///my_data1.db
Done.


FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude
6,2010-06-04,Falcon 9,6123.547647058824,LEO,CCSFS SLC 40,None None,1,0,0,0,None,1.0,0,B0003,-80.577366,28.5618571
8,2012-05-22,Falcon 9,525.0,LEO,CCSFS SLC 40,None None,1,0,0,0,None,1.0,0,B0005,-80.577366,28.5618571
10,2013-03-01,Falcon 9,677.0,ISS,CCSFS SLC 40,None None,1,0,0,0,None,1.0,0,B0007,-80.577366,28.5618571
12,2013-12-03,Falcon 9,3170.0,GTO,CCSFS SLC 40,None None,1,0,0,0,None,1.0,0,B1004,-80.577366,28.5618571
13,2014-01-06,Falcon 9,3325.0,GTO,CCSFS SLC 40,None None,1,0,0,0,None,1.0,0,B1005,-80.577366,28.5618571


## 3. Payload Analysis

### Task 2: Calculate total payload mass for ISS missions

In [ ]:
# Calculate the total payload mass carried by boosters launched to ISS (NASA CRS missions)
%sql SELECT SUM(PayloadMass) AS 'Total Payload Mass (kg)' FROM SPACEXTBL WHERE Orbit = 'ISS'

 * sqlite:///my_data1.db
Done.


Total Payload Mass (kg)
68878.7


### Task 3: Calculate average payload mass for Falcon 9 boosters

In [ ]:
# Calculate average payload mass carried by Falcon 9 boosters
%sql SELECT AVG(PayloadMass) AS 'Average Payload Mass (kg)' FROM SPACEXTBL WHERE BoosterVersion = 'Falcon 9'

 * sqlite:///my_data1.db
Done.


Average Payload Mass (kg)
6123.547647058824


## 4. Landing Outcome Analysis

### Task 4: First successful ground landing date

In [ ]:
# Find the date when the first successful landing outcome on ground pad was achieved
%sql SELECT MIN(Date) AS 'First Successful Landing Date' FROM SPACEXTBL WHERE Outcome = 'True RTLS'

 * sqlite:///my_data1.db
Done.


First Successful Landing Date
2015-12-22


### Task 5: Successful drone ship landings with specific payload mass

In [ ]:
# List flight numbers with successful drone ship landings and payload mass between 4000-6000 kg
%sql SELECT FlightNumber FROM SPACEXTBL WHERE Outcome = 'True ASDS' AND PayloadMass BETWEEN 4000 AND 6000

 * sqlite:///my_data1.db
Done.


FlightNumber
29
33
38
49
67
71
85


### Task 6: Count of all mission outcomes

In [ ]:
# List the total number of successful and failure mission outcomes
%sql SELECT Outcome, COUNT(Outcome) AS 'Count' FROM SPACEXTBL GROUP BY Outcome

 * sqlite:///my_data1.db
Done.


Outcome,Count
False ASDS,6
False Ocean,2
False RTLS,1
None ASDS,2
None None,19
True ASDS,41
True Ocean,5
True RTLS,14


### Task 7: Flights with maximum payload mass

In [ ]:
# List all flight numbers that carried the maximum payload mass (using subquery)
%sql SELECT FlightNumber FROM SPACEXTBL WHERE PayloadMass = (SELECT MAX(PayloadMass) FROM SPACEXTBL)

 * sqlite:///my_data1.db
Done.


FlightNumber
84
87
89
90
92
93
95
96
100
102


## 5. Time-Based Analysis

### Task 8: Failed drone ship landings in 2015

In [ ]:
# List records showing month, outcome, flight number, and launch site for failed drone ship landings in 2015
%sql SELECT STRFTIME('%m', Date) AS 'Month', Outcome, FlightNumber, LaunchSite FROM SPACEXTBL WHERE Outcome = 'False ASDS' AND STRFTIME('%Y', Date) = '2015'

 * sqlite:///my_data1.db
Done.


Month,Outcome,FlightNumber,LaunchSite
01,False ASDS,19,CCSFS SLC 40
04,False ASDS,22,CCSFS SLC 40


### Task 9: Ranking landing outcomes by count (2010-2017)

In [ ]:
# Rank the count of landing outcomes between 2010-06-04 and 2017-03-20 in descending order
%sql SELECT Outcome, COUNT(Outcome) AS 'Count' FROM SPACEXTBL WHERE Date BETWEEN '2010-06-04' AND '2017-03-20' GROUP BY Outcome ORDER BY Count DESC

 * sqlite:///my_data1.db
Done.


Outcome,Count
None None,9
True ASDS,5
False ASDS,4
True RTLS,3
True Ocean,3
None ASDS,2
False Ocean,2


## 6. Summary

This analysis explored SpaceX launch data using SQL queries, revealing:

- **Launch Sites**: Three primary sites - CCSFS SLC 40, VAFB SLC 4E, and KSC LC 39A
- **Payload Mass**: Total ISS mission payload was 68,878.7 kg with an average of 6,123.55 kg per launch
- **Landing Success**: First successful ground landing (RTLS) occurred on 2015-12-22
- **Outcomes**: Various landing outcomes including successful ASDS (drone ship) and RTLS (ground pad) landings
- **Temporal Patterns**: Analysis of landing outcomes between 2010 and 2017 shows progression of landing capabilities

The data demonstrates SpaceX's evolution in rocket landing technology and mission success rates over time.